<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%AA%A8%EB%8D%B8%EB%A7%812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch import optim
from torch import optim

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 불러오기

In [5]:
pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct.csv").head(3)

,review,reply,star,star_t,star_q,star_d,food,store,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,5,5.0,5.0,5.0,직화 삼겹옛날도시락,['1988응답하라추억의옛날도시락 방이점'],1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,5,5.0,5.0,5.0,직화 삼겹옛날도시락 볶음김치,['1988응답하라추억의옛날도시락 방이점'],0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,5.0,5.0,5.0,직화 삼겹옛날도시락 우삼겹청국장찌개 대접밥,['1988응답하라추억의옛날도시락 방이점'],1


In [6]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct.csv")
cols = ['review','reply','sentiment']
train = train[cols]
train.isnull().sum()

review        0
reply        48
sentiment     0
dtype: int64

In [7]:
idx_null = train[train['reply'].isnull()].index
train = train.drop(idx_null, axis=0)
train.isnull().sum()

review       0
reply        0
sentiment    0
dtype: int64

In [8]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0     1813
1    35279
Name: sentiment, dtype: int64

In [9]:
print(len((train[train['sentiment'] == 0])))
len(train[train['sentiment'] == 0])/len(train[train['sentiment'] == 1])

1813


0.051390345531335924

### 오버샘플링(긍정->부정 데이터)

In [10]:
pd.read_csv(f"{DATA_PATH}train_gpt_9300.csv").head(3)

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0


In [11]:
train_gpt = pd.read_csv(f"{DATA_PATH}train_gpt_9300.csv")
train_gpt.isnull().sum()

gpt_review    0
gpt_reply     0
sentiment     0
dtype: int64

In [12]:
# 컬럼명 통일
rename_col = {'gpt_review': 'review', 'gpt_reply': 'reply'}
train_gpt.rename(columns=rename_col, inplace=True)
train_gpt

,review,reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0


In [13]:
# 합치기
train = pd.concat([train, train_gpt], axis=0).reset_index(drop=True)
train

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1
3,,저희 매장과 함께 편안한 시간 되셨을까요 1 고객을 생각하는 마음 2 더 나은 맛과...,1
4,냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이...,주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요 고객님의 행복한 시간에 저희...,1
...,...,...,...
46410,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0
46411,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0
46412,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0
46413,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0


In [14]:
train['review'] = train['review'].astype(str)
train['reply'] = train['reply'].astype(str)

### 여기까지 저장하기

In [ ]:
# train.to_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct_gpt.csv", index=False)

# 여기까지 완료된 데이터 불러오기

In [ ]:
# train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct_gpt.csv")
# train.head(3)

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1


In [15]:
train.isnull().sum()

review       0
reply        0
sentiment    0
dtype: int64

## 추가 전처리

### 개별 삭제(추론 결과보고 계속 추가)

In [24]:
train['reply'][14381]

'안녕하세요 부잣집 주문해 주셔서 감사합니다 바쁘신 와중에 별 5 리뷰 감사드립 다 양도 맛도 맘에 드셨다니 너무 다행입니다 앞으로 더 맛나게 끓여드릴게요 건강하게 오래오래 맛있고 푸짐한 음식 제공 드릴게요 언제나 맛과 양까지 다 잡도록 저희 부잣집은 항상 노력 하겠음 다 꼭 생각 나시면 부잣집 찾아주세요'

In [22]:
print(train[train['reply'].str.contains("부잣집")].index)

Int64Index([14380, 14381, 14382, 14383, 14384, 14385, 14386, 14387, 14388,
            14389,
            ...
            14527, 14528, 14529, 14530, 14531, 14532, 14533, 14534, 14535,
            14536],
           dtype='int64', length=157)


In [25]:
remove_keywords = ['직구 쌈','직구 삼','빕스','계림','부잣집']
for keyword in remove_keywords:
    train['reply'] = train['reply'].str.replace(keyword, '')

train['reply'][32990]

'저희 을 이용해 주셔서 감사합니다 별 다섯 개 리뷰 감사합니다 빠른 배달 맛있는 음식 유지하도록 노력하고 있습니다 앞으로도 정성을 다해 보답하겠습니다'

### 단문 데이터 제거
- 데이터 정제 과정에서 무의미해진 데이터는 학습데이터로 사용불가
    - 리뷰 : 공백만 생겼거나, 리뷰 이벤트를 위한 단답형 제거
    - 답변 : 16자 이하 답변 제거

In [26]:
len(train['review'][3]) , len(train['reply'][642])

(1, 3)

In [27]:
train['review'][3] , train['reply'][642]

(' ', '이렇게')

In [28]:
train[(train['review'].str.len() > 4) & (train['review'].str.len() <= 10)] # 리뷰

,review,reply,sentiment
24,언제나 맛있습니다,소중한 시간에 저희 매장을 다시 이용해 주셔서 감사합니다 만족스러우셨던 거 같아 다...,1
26,잘 먹었습니다,좋은 리뷰를 남겨주셔서 감사해요 저희 팀은 여러분이 웃을 수 있게 항상 최선을 다하...,1
30,양도 많고 맛있어요,잊지 않으시고 리뷰 남겨주셔서 정말 감사합니다 맛있는 메뉴들은 양껏 먹어도 먹고 싶...,1
40,맛있게 잘 먹었어요,이렇게 고객님을 뵙게 되어 너무 영광 준비한 저희 메뉴가 입맛에는 맞으실까 항상 걱...,1
54,양 많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지 감동 고객님들의 배고픔을 저희는 지켜만 ...,1
...,...,...,...
42943,너무 달았어요,단맛이 강한 음식이 었습니다. 앞으로는 맛을 조절하여 제공하도록 노력하겠습니다.,0
43100,맛이 별로였어요,"죄송해요, 많이 실망하셨군요. 저희 음식이 맛있게 전달되도록 조금 더 노력하겠습니다.",0
44170,맛이 별로였어요,"죄송합니다, 맛에 대해 별점을 받으셨다니 정말 안타깝습니다. 더 맛있는 음식으로 보...",0
44460,맛이 별로였어요.,죄송합니다. 맛이 별로였다니 정말 안타깝네요. 다음에는 더 좋은 맛으로 보답하겠습니다.,0


In [29]:
train[(train['reply'].str.len() > 15) & (train['reply'].str.len() <= 17)] # 답변

,review,reply,sentiment
556,맛있어요 감사 허무니 당,네넵 감사 감사 감사 허무니 당,1
561,배달도 예상시간보다 빨리 왔고 치즈도 엄청 많이 왔어요 진짜 맛있습니다,양도 맛도 항상 진심 감사합니다,1
651,오오 떡볶이도 진짜 맛있고 튀김도 맛있네요,감사합니다 자주 이용해 주세요,1
685,아쉽게 사진을 찍진 못했지만 맛있게 잘 먹었어요,맛있게 드셨다니 너무 감사합니다,1
688,양도 푸짐하고 맛도 괜찮네요,감사해요 또 만났으면 좋겠습니다,1
...,...,...,...
34759,맛있게 잘 먹었어요,님 맛있게 드셔주셔서 감사합니다,1
36305,맛있었어요 커피도 안 흐르고 잘 왓어옹,맛있게 드셨다니 제가 감사합니다,1
37889,숙주가 많아서 싫어요. 맛도 별로였어요.,불편을 드려서 정말 죄송합니다.,0
38580,맛이 별로였어요. 육수도 많이 없고 갈비도 말라서 실망했어요.,불편을 드려 정말 죄송합니다.,0


In [30]:
len(train[train['review'].str.len() <= 9]) , len(train[train['reply'].str.len() <= 17])

(2619, 974)

In [31]:
train = train[(train['review'].str.len() > 9) & (train['reply'].str.len() > 17)].reset_index(drop=True)
train

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1
3,냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이...,주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요 고객님의 행복한 시간에 저희...,1
4,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께해 주셔서 영광 다음에도 고객님의 즐거운 순간을 함...,1
...,...,...,...
42918,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0
42919,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0
42920,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0
42921,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0


## 긍부정 데이터 비율 선택

In [32]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0    10921
1    32002
Name: sentiment, dtype: int64

In [33]:
train_neg = train[train['sentiment'] == 0]

# 긍정 데이터 중 중복된 답변제거
train_pos = train[train['sentiment'] == 1].drop_duplicates(subset='reply', keep=False)
train_pos['sentiment'].value_counts().sort_index()

1    14903
Name: sentiment, dtype: int64

In [34]:
length = len(train_neg)
length

10921

In [35]:
# 긍정 데이터를 랜덤으로 추출하기 위해 섞기
shuffled_train_pos = train_pos.sample(frac=1, random_state=42)
train_pos = shuffled_train_pos.iloc[:length].reset_index(drop=True) # 1:1 비율
# train_pos = shuffled_train_pos.iloc[:length*2].reset_index(drop=True) # 2:1 비율 만들기

# 데이터 합치기
train = pd.concat([train_neg, train_pos], ignore_index=True)
train['sentiment'].value_counts().sort_index()

0    10921
1    10921
Name: sentiment, dtype: int64

In [36]:
# 다시 섞기
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
train

,review,reply,sentiment
0,양이 너무 적고 서비스도 별로였어요.,양이 부족했고 서비스도 만족스럽지 않았다니 정말 죄송합니다. 앞으로 더 나은 서비스...,0
1,찰밥이 맛이 좋네요 잘 먹었습니다,맛있게 드셔주셔서 감사드립니다 늘 건강하시고 내일은 오늘보다 더 행복한 하루 보내시...,1
2,잘 먹었어요 메인인 고기 양이 사이드가 엄청 많네요,죄송하네요 고기 양이 적어서요 보쌈정식 1인분이 8000천윈,1
3,"가격이 너무 비싸요, 그만큼 맛이 안 나와서 아쉬웠어요","안녕하세요, 가격에 대해서 죄송합니다. 맛에 대한 기대치에 맞지 못했던 것 같아 아...",0
4,음식이 완전히 맛이 없었어요. 정말 실망스러웠어요,저희 식당의 음식이 기대에 못 미치셨다니 정말 안타깝습니다. 더 맛있는 음식을 제공...,0
...,...,...,...
21837,배달도 신속하고 맛도 있어서 좋습니다,감사합니다 맛있게 드셨다니 제가 더 감사합니다 지금처럼 맛있는 배달이 빠른 되겠습니다,1
21838,항상 언제나 맛있게 잘 먹고 있어요,항상 맛있게 드셔주셔서 감사합니다 변함없는 맛으로 보답해 드리는 되겠습니다 요즘 날...,1
21839,맛이 기대한 만큼 좋지 않았어요. 다른 집에서 시킬 걸 그랬다는 생각이 들어요,맛에 실망하셨다니 저희가 더 만족스러운 경험을 드릴 수 있도록 노력하겠습니다.,0
21840,가격 대비 좀 심함,돈코쓰 세트가 제일 인기 많은 메뉴인데 많이 부족했나 봅니다 죄송합니다,0


# 모델 불러오기

In [37]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00


In [38]:
model_name = "skt/kogpt2-base-v2"

In [39]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [40]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [41]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 데이터셋 만들기

In [42]:
train.head()

,review,reply,sentiment
0,양이 너무 적고 서비스도 별로였어요.,양이 부족했고 서비스도 만족스럽지 않았다니 정말 죄송합니다. 앞으로 더 나은 서비스...,0
1,찰밥이 맛이 좋네요 잘 먹었습니다,맛있게 드셔주셔서 감사드립니다 늘 건강하시고 내일은 오늘보다 더 행복한 하루 보내시...,1
2,잘 먹었어요 메인인 고기 양이 사이드가 엄청 많네요,죄송하네요 고기 양이 적어서요 보쌈정식 1인분이 8000천윈,1
3,"가격이 너무 비싸요, 그만큼 맛이 안 나와서 아쉬웠어요","안녕하세요, 가격에 대해서 죄송합니다. 맛에 대한 기대치에 맞지 못했던 것 같아 아...",0
4,음식이 완전히 맛이 없었어요. 정말 실망스러웠어요,저희 식당의 음식이 기대에 못 미치셨다니 정말 안타깝습니다. 더 맛있는 음식을 제공...,0


In [43]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        sentiment = item["sentiment"]
        review = item["review"]
        answer = item["reply"]

        return {
            "sentiment": sentiment,
            "review": review,
            "answer": answer
        }

In [44]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [f"<q>{item['review']}</s><a>{item['answer']}</s>" for item in batch]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [45]:
dt = ChatDataset(train)
dt[0]

{'sentiment': 0,
 'review': '양이 너무 적고 서비스도 별로였어요. ',
 'answer': '양이 부족했고 서비스도 만족스럽지 않았다니 정말 죄송합니다. 앞으로 더 나은 서비스를 제공하도록 노력하겠습니다. '}

In [46]:
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405, 13692, 12371, 19157, 10997,  7235, 15126,  8041,
           8006, 25856,   739,     1,  9724,   439,   405, 13692, 10667, 10792,
          10997,  7235, 13902, 46399, 10171, 16179, 29205, 11482,  7832, 37194,
          12634,  9267, 27705, 13796, 10312,  9825, 10805, 26616, 16691,   739,
              1,     3,     3,     3,     3,     3,     3,     3,     3,     3],
         [ 9724,   455,   405,  8339,  7605,  8146, 15882,  9677,  7098,  8084,
           9443,  9784,  8017, 16913,  7182,     1,  9724,   439,   405,  7496,
          18479,  9535,  7810,  8236,  7810,  7788, 15940,  7281,  7483,  7172,
           7182, 10061, 11342, 28944,  6889,  9094, 16373, 10070,  9518,  9267,
          29918, 12557, 11219,  7888,  6962,  9117,  7377,  7172,  7182,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,

In [47]:
model(**batch["x"]).logits.shape

torch.Size([2, 50, 51200])

In [48]:
batch["sentiment"].to(device, dtype=torch.float32)

tensor([0., 1.], device='cuda:0')

# 동결

In [49]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [50]:
for name,param in model.named_parameters():
    print(name,param)
    break

transformer.wte.weight Parameter containing:
tensor([[ 0.0282, -0.0369, -0.0061,  ..., -0.0248, -0.0068,  0.0108],
        [ 0.0104,  0.0134,  0.0427,  ...,  0.0091, -0.0053, -0.0012],
        [ 0.0314, -0.0164,  0.0178,  ..., -0.0054, -0.0480,  0.0040],
        ...,
        [-0.0244, -0.0089, -0.0003,  ...,  0.0009,  0.0304, -0.0017],
        [-0.0206,  0.0045,  0.0241,  ...,  0.0630, -0.0192,  0.0099],
        [-0.0199, -0.0382,  0.0268,  ...,  0.0299, -0.0154, -0.0423]],
       requires_grad=True)


## 일부 동결

In [51]:
for name,param in model.named_parameters():
    if not name.startswith("lm_head"):
        param.requires_grad = False
        if name.startswith("transformer.h.3"): # 1/4 동결
            break

In [52]:
for name,param in model.named_parameters():
    print(name,param.requires_grad) # lm_head 만 가중치 업데이트

transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

## 전체 동결

In [ ]:
# class FreezeNet(torch.nn.Module):
#     def __init__(self, model):
#         super().__init__()
#         self.model = model
#         for param in self.model.parameters():
#             param.requires_grad = False
#         self.model.lm_head = torch.nn.Linear(768,51200)

#     def forward(self,x):
#         return self.model(x)

In [ ]:
# model = FreezeNet(model)
# for param in model.parameters():
#     print(param.requires_grad)

# 학습루프

In [53]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        # x = {key: val.to(device, dtype=torch.float32) for key, val in batch["x"].items()}
        # sentiment = batch["sentiment"].to(device, dtype=torch.float32)
        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        # batch["x"]["input_ids"].requires_grad = True
        # batch["x"]["attention_mask"].requires_grad = True

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [ ]:
reset_seeds(SEED)
batch_size = 2
epochs = 10

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

## 동결 코드 ##
# 일부 동결 #
for name,param in model.named_parameters():
    if not name.startswith("lm_head"):
        param.requires_grad = False
        if name.startswith("transformer.h.3"): # 1/4 동결
            break

# # 모델의 특정 레이어를 동결 해제
# for layer in model.transformer.h:  # 예를 들어, 모델의 트랜스포머 레이어를 동결 해제
#     for param in layer.parameters():
#         param.requires_grad = True
#         param.data = param.data.to(torch.float32)

# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-5)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(train)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")


  0%|          | 0/11080 [00:00<?, ?it/s]

1번째 epoch: 2.2781


  0%|          | 0/11080 [00:00<?, ?it/s]

2번째 epoch: 1.9387


  0%|          | 0/11080 [00:00<?, ?it/s]

...

8번째 epoch: 1.3143


  0%|          | 0/11080 [00:00<?, ?it/s]

9번째 epoch: 1.2332


  0%|          | 0/11080 [00:00<?, ?it/s]

10번째 epoch: 1.1584


- 0905_csv 5000개 epochs = 10, 0.6046
- 0906_csv sentiment:0(부정) batch = 2, epochs = 20, 0.4039 (18->0.3136)
- 0906_csv sentiment:1(긍정) batch = 2, epochs = 10, 0.4291
- 0907_df_neg_augmented.csv batch = 2, epochs = 15, 0.2129(2724개)
- 0908_ng_df.csv batch = 2, epochs = 18, 0.2032(1483개)
- 0908_ps_df.csv batch = 2, epochs = 15, 0.2152(3000개)
- gpt_net_reviews_4400(부정) + train+sentiment_0913(긍정) batch = 2, epochs = 10, 0.4319(8781개)

# 학습모델 저장

In [ ]:
model.save_pretrained(f"{DATA_PATH}0908_gpt_freeze_all")

# 저장한 모델 로드

In [54]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0908_gpt_freeze_all").to(device)

## 테스트루프

In [ ]:
# @torch.no_grad()
# def chatbot(model, tokenizer, max_len, device, num_samples=3):
#     model.eval()
#     while True:
#         text = input("user > ").strip()
#         if text == "quit":
#             break

#         text = "<q>" + text + "</s><a>"
#         x = tokenizer.encode(text, return_tensors="pt").to(device)
#         q_len = len(text) + 1

#         generated_texts = [] # 답변 후보군을 담을 리스트
#         for _ in range(num_samples):
#             result_ids = model.generate(x,
#                                         max_length=max_len,
#                                         repetition_penalty=2.0,
#                                         use_cache=True,
#                                         do_sample=True,
#                                         temperature=0.9,
#                                         top_k=50,
#                                         num_return_sequences=1
#                                         )
#             generated_text = tokenizer.decode(result_ids[0])
#             generated_text = generated_text[q_len:-4]
#             generated_texts.append(generated_text)

#         print("bot >")
#         for i, generated_text in enumerate(generated_texts, 1):
#             print(f"{i}. {generated_text}")
#         print("--------------------------------------------------------------------------------------------")

In [55]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f56493c8c9f22d3c8aa9af7c2b421362bfc9be5196d99be7992c1cfda1460f35
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [82]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text, alpha=0.9): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))
    # jaccard_sim = jaccard_score(input_tokens, generated_tokens)

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [83]:
evaluate_similarity("맛은 있는데 양이 적어요",'맛있게 드셔주셔서 감사합니다. 다음에 양 많이 드릴게요')

0.35823020339012146

In [77]:
# 옵션
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [93]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=10):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.8,
                                        top_k=50,
                                        num_return_sequences=1
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            generated_text = paraphrase_text(generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [94]:
# 0908_gpt_freeze
chatbot_with_evaluation(loaded_model,tokenizer,128,device)

user > 맛은 있는데 양이 너무 적어요
(0.17970719933509827, '항상 준비된 신선한 재료로 최선을 다해 노력하겠습니다, 다음에도 더 좋은 요리로 찾아 뵐게요.')
(0.14930087327957153, '항상 노력하는 모습으로 고객님을 만나도록 노력하겠습니다. 더 좋은 요리와 서비스로 보답하겠습니다.')
(0.12196733951568604, '오늘 하루도 수고 많으셨고 내일도 행복한 일만 가득하시길 기원하겠습니다. 감사합니다.')
(0.22153555154800417, '항상 고객님의 행복한 식사시간을 위해 노력하겠습니다. 저희가 48시간 동안 푹 고아 낸 육수와 그날 잡은 신선한 고기로 만든 육수로 정말 맛있게 요리하겠습니다.')
(0.31574648022651675, '잘 먹었다고 내일 인사해야지')
(0.2833652436733246, '잘 팔리길 바라는 마음으로 맛있는 요리 만들어 드리고 다음에도 또 찾아 뵐게요 감사합니다.')
(0.5509728312492371, '항상 양의 측면에서 균형을 잡아 드리려고 노력하는데, 이번에는 맛이 좀 약했다고 연락을 받았군요.')
(0.2922369182109833, '이용해 주셔서 대단히 감사합니다. 앞으로도 더 좋은 요리로 찾아 뵙겠습니다.')
(0.15353254079818726, '더 많은 만족을 위해 노력하며, 고객님들의 불편함을 해결하기 위해서는 항상 품질에 집중할 것이며, 앞으로 더욱 발전하도록 하겠습니다.')
(0.25164613723754886, '자주 찾아 주셔서 감사합니다. 앞으로도 더 좋은 요리와 다양한 정보 부탁드립니다.')

Best_reply > 항상 양의 측면에서 균형을 잡아 드리려고 노력하는데, 이번에는 맛이 좀 약했다고 연락을 받았군요.
Best_similarity_score: 0.5509728312492371
--------------------------------------------------------------------------------------

- 0914_trainsentiment_0913+gpt

In [ ]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님의 의견은 저희에게 소중한 자산입니다. 계속해서 피드백을 기다리고 있겠습니다. 좀 더 나은 서비스를 제공할 수 있는 방법을 찾아보도록 하겠습니다,, 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요
bot > 고객님께서 불편을 느끼셨다니 죄송합니다. 좀 더 신속한 배달로 적절한 조치를 취하도록 하겠습니다., 
--------------------------------------------------------------------------------------------
user > 먹다가 머리카락이 나왔어요
bot > 맛있게 드셨다니 감사합니다! 언제나 최선을 다하여 최고의 음식을 드리는 땡초가 되겠습니다, 고객님의 소중한 리뷰에 항상 귀기울이는 벼락으로 보답드릴게요, 자주 애용해주세용~ 신메뉴 혹시라도 배달 받으신다면 잊지 않고 부탁드려요. 적극 반영하겠습니다. 이번주는 급격히 발전하는 한주입니다. 폭염 속 건강 조심하시고 행복하십쇼입니다. 저희 메뉴가 많이 있으니 이용바랍니당 다음에 또 찾아 주실거죠? 감사합니다. 
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉬워요
bot > 고객님의 의견을 반영하여 더 나은 제품을 만들기 위해 노력하겠습니다. 불편을 드려 죄송합니다., 
--------------------------------------------------------------------------------------------
user > 여기 피자가 우리 동네에서 제일 맛있어요,, 쿠폰도 자주 줘서 너무 좋아요
bot > 고객님 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가

- 0905_kogpt2_chat

In [ ]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님 ~! 먼저 죄송합니다. 부족한 부분으로 실망시켜  사과드리며 늘 저희를 사랑해 주시는 고객님이시군요 정성스러운 사진 리뷰와도 너무 감사한데, 먹음직스럽게 찍어주신 소중한 말씀까지.
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 소중한 리뷰 감사드립니다 저희 매장은 맛과 양, 서비스 등 모든 면에서 최고가 되기 위해 항상 최선을 다하고 있습니다, 언제든지 부족함 없이 드실 수 있도록 준비해 보내드리겠습니다.! 앞으로도 만족만 드릴 만반의 준비를 하여 계속 마음속에 남아 있고 싶거든요 <s>~ 좋은 하루 되세요. 고객님 ~
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 오늘 하루를 시작하며 고객님의 리뷰을 보고 웃음 짓게 됩니다 다음에도 점심 식사를 기분 좋게 드실 수 있길 바랍니다. 저희는 항상 맛있는 음식 제공에 힘내며 언제라도 생각나는 맛집이 되도록 노력하겠습니다, 앞으로도 더욱 만족스러운 식사하실 땐 더 큰 기쁨으로 인사드리겠습니다.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 맛있게 드셨다니 다행입니다! 찾아주시는 피드백은 언제든 잘 수렴하고 이를 통해 나날이 발전하는 곳이 되도록 하겠습니다, 더욱더 좋은 모습으로 찾아가겠습니다.<s>. 기쁨 가득한 하루 보내세요 <s>, 감사해요.<s><s><s><s><s> 소중한 리뷰에 제가 느낀 이 행복만큼 고객님도 항상 노력하시길 바라며 오늘

- 0906_negative_kogpt2_chat

In [ ]:
chatbot(model,tokenizer,128,device)

user > 맛이 없어요
bot > 안녕하세요! 저희 맛집이 만족한 식사시간이 되지 못해서 죄송합니다.. 양이 부족하다.?
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 고객님 식사에 불편을 드려 너무 죄송합니다 다음엔 더욱 신경 써서 만족하실 수 있도록 전달하겠습니다, 감사합니다.<s> <unk>얌꿍이 약간 매콤한 맛으로 바뀌어서요~ 더 맛있는 요리로 찾아뵐게요. 가성비 있고 푸짐한 양 담아 제공하는 팔당이 되겠습니다.<s> <b>이상, 당일 납품받는 재료로 최고의 신선도를 유지하여 드리는 회사가 되도록 노력할 것이 무엇보다 중요한데도 불구하고 저희 음식 포장 문제가 발생을 하니 정말이지에서 누락되는 경우가 발생되고 있네요, 다시 한
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 고객님들께 만족감을 드리려고 연구하고 노력하고 있는데 아직 많이 부족한 것 같습니다 시간 내시어나 말씀해 주시면 감사하겠습니다.<s> <blight difference in size of the nice wit must have big chance, I recommend you try little phortfyetangnam. HMT 4. 콜라 1.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot >  양적으로 실망하셨다니 너무 죄송합니다. 다음번엔 더욱 맛있는 음식으로 보답 드리도록 하겠습니다.<s> 감사합니다! <s> 사진은 다시 한번 더 수정해서 올리거나

- 0906_positive_kogpt2_chat

In [ ]:
chatbot(pos_model,tokenizer,128,device)

user > 맛이 없어요
bot >  안녕하세요! 푸짐하고 맛있는 헬로 팬입니다 맛있게 잘 드셔주신 것 같아 기분 좋네요. 앞으로도 변함없는 모습으로 보답하는헬로, 닭갈비 생각나는 날 되겠습니다, 다음에 또 만나 <unk> 주세요 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 안녕하세요! 소중한 별점 만 점 리뷰 감사합니다 즐거운 식사가 되신 거 같아 너무 뿌듯해요, 행복하게 보내셔서 진심으로 감동의 물결이 휘몰아치네요. 기분 좋은 칭찬 사진 너무나도 이쁘게 찍어주셨는데 맛있게 드시고 예쁜 포토리뷰까지 남겨 주시니 그저 아쉽지 않습니다, 이렇게 받은 신뢰에 꼭 보답할 수 있도록 매일 신선한 재료로 정성을 다하겠습니다.
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가 남겨주셔서 너무 감사해 앞으로도 신선한 재료 아끼지 않고 정성을 가득 담아 준비 약속드릴게요, 무더운 날씨 항상 더위 조심하시고 늘 행복만 전해드리겠습니다! 
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 안녕하세요~ 소중한 시간 내셔서 리뷰까지 감동입니다! 고객님께서 남겨주신 별점에, 저희를 춤추게 합니, 기분이 좋습니다, 항상 힘내어 드실 수 있도록 꾸준히 노력하겠습니다. 앞으로도 행복한 하루 보내시길 바랄 테니 많이 애용해 주시고, 감기 조심하시고 파이팅 하오 
-------------------------------